In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# my lib
import sys
sys.path.append(r'E:\01_PProj\ML_LIB')
sys.path.append(r'D:\00_work\ML_LIB')
import Lib_sjw.training as tr
import Lib_sjw.model_interface as mi
import Lib_sjw.model_parmas as mp
import Lib_sjw.evaluator as ev
import Lib_sjw.classification_util as cu
import Lib_sjw.ensemble as es
import Lib_sjw.todataframe as td
import Lib_sjw.eda as eda

import pandas as pd
import numpy as np

In [29]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split , KFold


In [17]:
data = load_iris()

x = data.data
y = data.target
cols = data.feature_names

df = pd.DataFrame(x , columns = cols )

In [23]:
pd.qcut(df['sepal length (cm)'], 4 , [0,1,2,3])

0      0
1      0
2      0
3      0
4      0
      ..
145    3
146    2
147    3
148    2
149    2
Name: sepal length (cm), Length: 150, dtype: category
Categories (4, int64): [0 < 1 < 2 < 3]

In [24]:
df['target'] = y

In [30]:
def fit_transform( feature_names ,df_train, test_data,target,alpha=5, add_random=False, rmean=0, rstd=0.1, folds=4):
    target_col = 'target'
    train_data = df_train.copy()
    train_data [target_col] = target
    columns = feature_names
    encoded_cols = []
    #self.target_mean_global = train_data[target_col].mean()
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()
        target_means_cats = train_data.groupby(col)[target_col].mean()
        target_means_cats_adj = (target_means_cats*nrows_cat + 
                                 target_mean_global*alpha)/(nrows_cat+alpha)
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)
        # Getting a train encodings
        
        kfold = KFold( folds, shuffle=True, random_state=1)
        parts = []
        for tr_in, val_ind in kfold.split(train_data[target_col].values):
            # divide data
            df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
            # getting means on data for estimation (all folds except estimated)
            nrows_cat = df_for_estimation.groupby(col)[target_col].count() # 각 카테고리별 샘플 수 
            target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
            target_means_cats_adj = (target_means_cats*nrows_cat + target_mean_global*alpha)/(nrows_cat+alpha)
            encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, 
                                                                         size=(encoded_col_train_part.shape[0]))
            # Saving estimated encodings for a fold
            parts.append(encoded_col_train_part) # 1개의 폴드에 대한 엔코딩값
        encoded_col_train = pd.concat(parts, axis=0) # 모든 폴드에 대한 엔코딩 값
        encoded_col_train.fillna(target_mean_global, inplace=True) # 빠진 값은 글로벌 평균으로 채워넣기 
       
        # Saving the column with means
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = target_mean_global
        encoded_cols.append(  pd.DataFrame({col + '_' + 'mean_'+target_col : encoded_col})  ) # 리스트, 안쪽은 1컬럼짜리 시리즈
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], all_encoded.loc[test_data.index, :])

In [31]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

In [24]:
df['sepal length (cm)'] = pd.qcut(df['sepal length (cm)'], 4 , [1,2,3,4])
df['sepal width (cm)'] = pd.qcut(df['sepal width (cm)'], 4 , [0,1,2,3])

In [25]:
xtrain, xtest , ytrain , ytest = train_test_split( df , y )

In [34]:
a , b = fit_transform( ['sepal length (cm)' , 'sepal width (cm)'], xtrain  , xtest , ytrain)

In [38]:
print(xtrain.shape , a.shape)

(112, 4) (112, 2)


In [36]:
b

,sepal length (cm)_mean_target,sepal width (cm)_mean_target
26,0.322216,0.385657
129,1.575206,1.131845
135,1.575206,1.131845
90,0.845009,1.325116
47,0.322216,0.958214
10,0.845009,0.385657
37,0.322216,0.385657
136,1.392587,0.385657
1,0.322216,1.131845
110,1.575206,0.958214
